# Tutoriel pytorch - TP3 - IFT725

Tel que mentionné dans l'énoncé du travail, vous devez recopier les blocs de code du tutoriel suivant

https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

en donnant, pour chaque bloc, une description en format "markdown" de son contenu.

# Tenseurs
## Avec Numpy

In [ ]:
import numpy as np

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

# Create some random data.
x = np.random.randn(N, Din)
y = np.random.randn(N, Dout)

w1 = np.random.randn(Din, H)
w2 = np.random.randn(H, Dout)

lr = 1e-6
for e in range(500):
    # Forward Pass
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute data
    loss = np.square(y_pred - y).sum()
    print(e, loss)

    # Backward Pass
    grad_y_pred = 2.0* (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Weight update
    w1 -= lr * grad_w1
    w2 -= lr * grad_w2

Ce bloc permet de créer un réseau avec une seule couche pleinement connectée asssociée à fonction d'activation ReLU.

Pour cela, on s'intéresse à plusieurs variables essentielles:
x est un ndarray contenant les données d'entrée du modèle, dont le ndarray y représente ses labels.
w1 et w2 contiennent les poids que le modèle apprend, initialisés aléatoirement. Ceux sont également des ndarrays, qui 
sont mis à jour à chaque epoch.
Le taux d'erreur du modèle est contenu dans la variable loss.
grad_w1 et grad_w2 sont les résultats des calculs des gradients des poids du réseau.

## Via Pytorch

In [ ]:
import torch

dtype = torch.float
device = torch.device("cpu")       # gpu = cuda

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

w1 = torch.randn(Din, H, device=device, dtype=dtype)
w2 = torch.randn(H, Dout, device=device, dtype=dtype)

lr = 1e-6
for e in range(500):
    # Forward pass 
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    loss = (y_pred - y).pow(2).sum().item()
    if e % 100 == 0:
        print(e, loss)

    # Backward pass
    grad_y_pred = 2.0* (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Weight update
    w1 -= lr * grad_w1
    w2 -= lr * grad_w2

Ce bloc permet de créer le même réseau que précédemment seulement au lieu d'utiliser des variables ndarrays ceux sont 
des tenseurs Pytorch. Ceux-ci ont quasiment la même structure qu'un ndarray mais permettent d'utiliser les ressources
d'un GPU dans le but d'accélérer les opérations matricielles.

# Autograd
## Tenseurs et autograd

In [ ]:
import torch

dtype = torch.float
device = torch.device("cpu")       # gpu = cuda

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

w1 = torch.randn(Din, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, Dout, device=device, dtype=dtype, requires_grad=True)

lr = 1e-6 
for e in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if e % 100 == 0:
        print(e, loss.item())
    
    loss.backward()


    with torch.no_grad():
        w1 -= lr* w1.grad
        w2 -= lr* w2.grad

        w1.grad.zero_()
        w2.grad.zero_()

Ce bloc utilise le package autograd, qui permet d'utiliser un graphe computationnel. Celui-ci est créé à la ligne
138, à la volée. Afin que les gradients des poids du réseau soient calculés lors de la rétro-propagation, on indique 
lors de l'initialisation des variables w_1 et w_2 qu'il est requis de sauvgarder leur gradient. 
La méthode backward permet de calculer la rétro rétropropagation et dès lors, le graphe créé précédemment est éliminé de
 la mémoire. 

## PyTorch: Définition de nouvelles fonctions autograd

In [ ]:
import torch 

class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """
    
    @staticmethod
    def forward(context, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        context.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(context, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = context.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("cpu")       # gpu = cuda

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

w1 = torch.randn(Din, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, Dout, device=device, dtype=dtype, requires_grad=True)

lr = 1e-6 
for e in range(500):
    relu = MyReLU.apply
    y_pred = relu(x.mm(w1)).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if e % 100 == 0:
        print(e, loss.item())

    loss.backward()

    with torch.no_grad():
        w1 -= lr * w1.grad
        w2 -= lr * w2.grad

        w1.grad.zero_()
        w2.grad.zero_()

Ce bloc définit tout d'abord une classe MyReLU permettant d'implémenter des fonctions propres autograd, forward et 
backward. 
La variable relu contient une instance de cette classe. La ligne 217 applique la méthode forward pour chacun des noeuds
du graphe. 

# nn module
## PyTorch: nn


In [ ]:
import torch
from torch.nn import *

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

criterion = MSELoss(reduction='sum')

# Sequential is a module which contains other modules, and applies them in sequence
# to produce its output. Each Linear Module is a Fully Connected Layer and holds 
# internal Tensor for its weights and bias.
model = Sequential(
    Linear(Din, H),
    ReLU(),
    Linear(H,Dout)
)

lr = 1e-6 
for e in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if e % 100 == 0:
        print(e, loss.item())
    
    model.zero_grad()
    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

Le package nn permet de disposer de différents Modules, notamment des couches du réseau. La variable model contient
donc plusieurs plusieurs Modules, possédant pour chacun d'eux leur propre implémentation de méthodes forward et backward
. Passer en paramètre des données d'entrée x à un Module couche effectue une propagation avant.
La variable criterion quant à elle est également un Module permettant de calculer l'erreur quadratique d'un modèle. 

## PyTorch: optim

In [17]:
import torch
from torch.optim import Adam
from torch.nn import *

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

lr = 1e-6
criterion = MSELoss(reduction='sum')
optimizer = Adam(model.parameters(), lr=lr)

for e in range(500):
    # Forward pass 
    y_pred = model(x)
    loss = criterion(y_pred, y)

    if e % 100 == 0:
        print(e, loss.item())

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

0 700.7799682617188
100 683.9097290039062
200 667.51123046875
300 651.6246337890625
400 636.155029296875


Ce bloc implémente l'utilisation du package optim permettant d'utiliser des méthodes d'optimisation de réseaux. 
La variable optimizer contient donc un type d'optimiseur qu'on applique sur les poids du réseau en appelant la méthode
step.

## PyTorch: Custom nn Modules

In [19]:
import torch
from torch.nn import *
from torch.optim import SGD

class TwoLayerNet(torch.nn.Module):
    def __init__ (self, Din, H, Dout):
        super(TwoLayerNet, self).__init__()
        self.linear1 = Linear(Din, H)
        self.linear2 = Linear(H, Dout)
    
    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        return self.linear2(h_relu)
    
# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

lr = 1e-6
criterion = MSELoss(reduction='sum')
optimizer = SGD(model.parameters(), lr=lr)

model = TwoLayerNet(Din, H, Dout)

for e in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if e % 100 == 0:
        print(e, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 639.8817138671875
100 639.8817138671875
200 639.8817138671875
300 639.8817138671875
400 639.8817138671875


Ce bloc implémente un réseau deux couches pleinement connectées prenant en entrée un Module. Cette classe 
réécrit les méthodes forward et backward du Module.         

## PyTorch: Control Flow + Weight Sharing


In [20]:
import random
import torch 

from torch.nn import *

class DynamicNet(torch.nn.Module):
    def __init__(self, Din, H, Dout):
        super(DynamicNet, self).__init__()
        self.input_linear = Linear(Din, H)
        self.middle_linear = Linear(H, H)
        self.output_linear = Linear(H, Dout)

    def forward (self, x):
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0,3)):
             h_relu = self.middle_linear(h_relu).clamp(min=0)
        return self.output_linear(h_relu)

# N is batch size, Din is input dimension, 
# H is hidden layer dimension and Dout is output dimension
N, Din, H, Dout = 64, 1000, 100, 10

x = torch.randn(N, Din, device=device, dtype=dtype)
y = torch.randn(N, Dout, device=device, dtype=dtype)

lr = 1e-6
criterion = MSELoss(reduction='sum')
optimizer = SGD(model.parameters(), lr=lr)

model = DynamicNet(Din, H, Dout)

for e in range(500):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if e % 100 == 0:
        print(e, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 615.9288330078125
100 663.4950561523438
200 663.4950561523438
300 624.8159790039062
400 615.9288330078125


Ce bloc implémente une classe DynamicNet qui permet de faire du partage des paramètres de poids du réseau grâce à 
sa réécriture de la méthode forward.
